In [1]:
from bs4 import BeautifulSoup
import requests
import json
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from urllib.parse import quote_plus

In [2]:
BASE_URL = 'https://www.aljazeera.net/'

articles = []
articlesTitles = []
articlesLinks = []
articlesDates = []
articlesContents = []

html = requests.get(BASE_URL + '/news/').text
soup = BeautifulSoup(html, "html.parser")

newsFeedContainer = soup.find('section', {'id': 'news-feed-container'})

for a in newsFeedContainer.find_all('a', href=True):
    articlesLinks.append(a['href'])
    articlesTitles.append(a.text)

for b in articlesLinks:
    html = requests.get(BASE_URL + b).text
    soup = BeautifulSoup(html, "html.parser")
    article = ''

    articleDate = soup.find('span', {'class': 'article-dates__published'}).text

    articlesParagraphsContainer = soup.find('div', {'class': 'wysiwyg wysiwyg--all-content css-1vkfgk0'})

    articlesParagraphs = articlesParagraphsContainer.find_all('p', recursive=False)

    articlesDates.append(articleDate)

    for p in articlesParagraphs:
        article += p.text + '\n'

    articlesContents.append(article)

for i in range(len(articlesTitles)):
    articles.append({
        'title': articlesTitles[i],
        'link': articlesLinks[i],
        'date': articlesDates[i],
        'content': articlesContents[i]
    })

In [3]:
with open('tmp/aljazzera.json', 'w', encoding='utf-8') as json_file:
    json.dump(articles, json_file, indent=4, ensure_ascii=False)

In [4]:
load_dotenv()

username = quote_plus(os.getenv('MONGODB_USERNAME'))
password = quote_plus(os.getenv('MONGODB_PASS'))
cluster = os.getenv('MONGODB_CLUSTER_ID')

uri = "mongodb+srv://" + username + ":" + password + "@cluster." + cluster + ".mongodb.net/?retryWrites=true&w=majority"

client = MongoClient(uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [5]:
db = client['nlp-lab-db']

collection = db['aljazeera-articles']

collection.insert_many(articles)

print('Articles inserted successfully')

Articles inserted successfully
